In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import joblib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score, roc_curve
)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score,confusion_matrix, classification_report, roc_curve, auc,f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import learning_curve, StratifiedKFold
from sklearn.exceptions import FitFailedWarning
from xgboost import XGBClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve, auc
)
import matplotlib.pyplot as plt


In [2]:
import pickle

import import_ipynb
from adda_boost import CustomAdaBoost
from random_forest2 import RFModel
from svm import CSVM
from XG_boost import XGBboost

In [8]:
import os

In [3]:
class WebLogModel():
    def __init__(self):
        with open("pickle/Addaboost.pkl","rb") as fd:
            self.AdaBoost_Classifier = pickle.load(fd) 
        
        with open("pickle/SVM.pkl","rb") as fd:
            self.SVM_Classifier = pickle.load(fd) 

        with open("pickle/RandomForest.pkl","rb") as fd:
            self.RF_Classifier = pickle.load(fd) 
        
        with open("pickle/XGBoost.pkl","rb") as fd:
            self.XGBoost_Classifier = pickle.load(fd) 

        print("models loaded")

    def predict(X):
        y_pred_adda_boost = self.AdaBoost_Classifier.predict(X)
        y_pred_svm = self.SVM_Classifier.predict(X)
        y_pred_rand_forest = self.RF_Classifier.predict(X)
        y_pred_XGB = self.XGBoost_Classifier.predict(X)

        all_y_pred = [y_pred_adda_boost,y_pred_rand_forest,y_pred_XGB,y_pred_svm]

        tot_votes = [0 for i in range(len_y)]

        for y_pred in all_y_pred:
            for i in range(len_y):
                tot_votes[i] += y_pred[i]

        y_pred_bagging = [1 if tot_votes[i]>=2 else 0 for i in range(len_y)]
        return y_pred_bagging

    def print_stats(y_test,y_pred):
        self.XGBoost_Classifier.print_stats(y_test,y_pred)
        self.XGBoost_Classifier.plot_confusion_matrix(y_test,y_pred)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

class CNN_Classifier:
    def __init__(self):
        self.model_path = "my_model.keras"
        self.model = load_model(model_path)

        model.summary()

    def predict(X):
        return self.model.predict(X)

In [6]:
session_cattagory = {}
def get_test_train_data_from_csv(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):

    df_merged = get_df_merged(hum_file,bot_file)

    X = df_merged.iloc[:, 1:-1].values
    Y = df_merged.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test,

def get_df_merged(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv"):
    df_bots = pd.read_csv(bot_file)
    df_bots["bot"] = 1
    df_bots
    for ses in df_bots.iloc[:,0].values:
        session_cattagory[ses] = 1

    df_hum = pd.read_csv(hum_file)
    df_hum["bot"] = 0
    df_hum
    for ses in df_hum.iloc[:,0].values:
        session_cattagory[ses] = 0
    df_merged = pd.concat([df_bots, df_hum], ignore_index=True, sort=False)

    return df_merged

def get_seesion_cat():
    df = get_df_merged()

    session_cat = {}
    for _,row in df[["session_id","bot"]].iterrows():
        session_cat[row[0]] = (row[1])

    return session_cat

#load test train data
X_train, X_test, y_train, y_test = get_test_train_data_from_csv("../parsed_data/human.csv","../parsed_data/bots.csv")
df = get_df_merged(hum_file = "../parsed_data/human.csv",bot_file = "../parsed_data/bots.csv")
session_catogory = get_seesion_cat()

C:\Users\sudin\AppData\Local\Temp\ipykernel_30656\4198039419.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  session_cat[row[0]] = (row[1])


In [ ]:

for batch in os.listdir("mv_data"):
    with open(os.path.join("mv_data",batch,"ses.txt"),"r") as fd:
        line = fd.readline()
        session_list = [x.strip().strip("'") for x in line.split(",")]
        print(lst[:5])

        data = np.load(dir +"/"+ np_file)
            ses_id= np_file[:-4]
            for key in data:
                X = data[key][...,np.newaxis]

['071tbv7fsev5d64kb0f9jieor6', '071tbv7fsev5d64kb0f9jieor6', '071tbv7fsev5d64kb0f9jieor6', '071tbv7fsev5d64kb0f9jieor6', '071tbv7fsev5d64kb0f9jieor6']
['cupcde8stlaqucrpahgnsiv0b2', 'cupcde8stlaqucrpahgnsiv0b2', 'cupcde8stlaqucrpahgnsiv0b2', 'cupcde8stlaqucrpahgnsiv0b2', 'cupcde8stlaqucrpahgnsiv0b2']
['t1eo5hqdtq21m7t0gulu2henl0', 't1eo5hqdtq21m7t0gulu2henl0', 't1eo5hqdtq21m7t0gulu2henl0', 't1eo5hqdtq21m7t0gulu2henl0', 't1eo5hqdtq21m7t0gulu2henl0']
['36rja5utab0pakj5136bi7rkop', '36rja5utab0pakj5136bi7rkop', '36rja5utab0pakj5136bi7rkop', '36rja5utab0pakj5136bi7rkop', '36rja5utab0pakj5136bi7rkop']
['6rm870g54tcodidn4igk93cg9t', '6rm870g54tcodidn4igk93cg9t', '6rm870g54tcodidn4igk93cg9t', '6rm870g54tcodidn4igk93cg9t', '6t1t0stce6afdl4ob3hl152fnj']
['akniekteebr1v6m9uetbh65sha', 'akniekteebr1v6m9uetbh65sha', 'akniekteebr1v6m9uetbh65sha', 'akniekteebr1v6m9uetbh65sha', 'akniekteebr1v6m9uetbh65sha']
['dft8qvdauujqen81pdjev9po2o', 'dft8qvdauujqen81pdjev9po2o', 'dft8qvdauujqen81pdjev9po2o', 'df

In [12]:
data = np.load("mv_data/batch_0/data.npz")

In [13]:
data

NpzFile 'mv_data/batch_0/data.npz' with keys: X, Y